In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db

In [2]:
# hack to get the path right
import sys
sys.path.append('..')

from ztf_maf.slicers import HealpixZTFSlicer

In [6]:
slicer = HealpixZTFSlicer(nside=64, lonCol='fieldRA', latCol='fieldDec')
#slicer = slicers.OpsimFieldSlicer()

metricList = []

m1 = metrics.IntraNightGapsMetric()
metricList.append(m1)
m2 = metrics.InterNightGapsMetric()
metricList.append(m2)
m3 = metrics.CountMetric(col='fieldRA')
metricList.append(m3)


filters = ['r','g']
programs = [1,2,3]

summaryMetrics = [metrics.MinMetric(), metrics.MeanMetric(), metrics.MaxMetric(), 
                  metrics.MedianMetric(), metrics.RmsMetric(), 
                 metrics.PercentileMetric(percentile=25), metrics.PercentileMetric(percentile=75)]

   
bDict={}
for i,metric in enumerate(metricList):

#    for j, filt in enumerate(filters):
#        sqlconstraint = "filter = '%s'"%(filt)
     for j, prog in enumerate(programs):
         sqlconstraint = "propID = '%s'"%(prog)
    
         bDict[sqlconstraint+metric.name] = metricBundles.MetricBundle(metric, slicer, sqlconstraint, 
                                              summaryMetrics=summaryMetrics)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [ ]:
outDir = 'ZTF_test'
ztfDB = db.OpsimDatabase('../sims/one_year_sim_incomplete.db',
                        defaultdbTables={'Summary':['Summary','obsHistID']})
ztfDB.raColName = 'fieldRA'
ztfDB.decColName = 'fieldDec'

resultsDb = db.ResultsDb(outDir=outDir)

bgroup = metricBundles.MetricBundleGroup(bDict, ztfDB, outDir=outDir, resultsDb=resultsDb)
bgroup.runAll()
bgroup.plotAll(closefigs=False)

Querying database with constraint propID = '3'
Found 15322 visits
Running:  ["propID = '3'Count fieldRA", "propID = '3'Median Intra-Night Gap", "propID = '3'Median Inter-Night Gap"]


In [5]:
for key in sorted(bDict):
    bDict[key].computeSummaryStats(resultsDb=resultsDb)
    print key, bDict[key].metric.name, bDict[key].summaryValues

propID = '1' Count fieldRA {'25th%ile': 38.0, 'Rms': 82.333984867768834, '75th%ile': 169.0, 'Min': 3.0, 'Max': 464.0, 'Median': 97.0, 'Mean': 109.54020007410152}
propID = '2' Count fieldRA {'25th%ile': 30.0, 'Rms': 21.048755416512282, '75th%ile': 62.0, 'Min': 1.0, 'Max': 149.0, 'Median': 46.0, 'Mean': 45.220394843129021}
propID = '3' Count fieldRA {'25th%ile': 12.0, 'Rms': 14.033336889114198, '75th%ile': 33.0, 'Min': 1.0, 'Max': 100.0, 'Median': 23.0, 'Mean': 23.521643240396326}
